In [55]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import plotly.express as px
from scipy import stats
%matplotlib inline

In [56]:
train_df = pd.read_csv('train.csv')
train_df

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0
1,1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0
2,2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0
3,3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0
4,4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
593989,593989,23004.26,0.152,703,20958.37,10.92,Female,Single,High School,Employed,Business,C3,1.0
593990,593990,35289.43,0.105,559,3257.24,14.62,Male,Single,Bachelor's,Employed,Debt consolidation,F5,1.0
593991,593991,47112.64,0.072,675,929.27,14.13,Female,Married,Bachelor's,Employed,Debt consolidation,C1,1.0
593992,593992,76748.44,0.067,740,16290.40,9.87,Male,Single,Bachelor's,Employed,Debt consolidation,B2,1.0


In [57]:
train_df.columns

Index(['id', 'annual_income', 'debt_to_income_ratio', 'credit_score',
       'loan_amount', 'interest_rate', 'gender', 'marital_status',
       'education_level', 'employment_status', 'loan_purpose',
       'grade_subgrade', 'loan_paid_back'],
      dtype='object')

In [58]:
test_df = pd.read_csv('test.csv')
test_df

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade
0,593994,28781.05,0.049,626,11461.42,14.73,Female,Single,High School,Employed,Other,D5
1,593995,46626.39,0.093,732,15492.25,12.85,Female,Married,Master's,Employed,Other,C1
2,593996,54954.89,0.367,611,3796.41,13.29,Male,Single,Bachelor's,Employed,Debt consolidation,D1
3,593997,25644.63,0.110,671,6574.30,9.57,Female,Single,Bachelor's,Employed,Debt consolidation,C3
4,593998,25169.64,0.081,688,17696.89,12.80,Female,Married,PhD,Employed,Business,C1
...,...,...,...,...,...,...,...,...,...,...,...,...
254564,848558,92835.97,0.068,744,29704.00,13.48,Female,Single,Bachelor's,Employed,Debt consolidation,B2
254565,848559,48846.47,0.091,634,20284.33,9.58,Female,Married,High School,Employed,Debt consolidation,D4
254566,848560,20668.52,0.096,718,26387.55,9.00,Male,Single,Master's,Employed,Debt consolidation,C4
254567,848561,34105.09,0.094,739,11107.36,9.81,Male,Single,Bachelor's,Employed,Business,C2


In [59]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

In [60]:
train_df.isna().sum()

id                      0
annual_income           0
debt_to_income_ratio    0
credit_score            0
loan_amount             0
interest_rate           0
gender                  0
marital_status          0
education_level         0
employment_status       0
loan_purpose            0
grade_subgrade          0
loan_paid_back          0
dtype: int64

In [61]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593994 entries, 0 to 593993
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    593994 non-null  int64  
 1   annual_income         593994 non-null  float64
 2   debt_to_income_ratio  593994 non-null  float64
 3   credit_score          593994 non-null  int64  
 4   loan_amount           593994 non-null  float64
 5   interest_rate         593994 non-null  float64
 6   gender                593994 non-null  object 
 7   marital_status        593994 non-null  object 
 8   education_level       593994 non-null  object 
 9   employment_status     593994 non-null  object 
 10  loan_purpose          593994 non-null  object 
 11  grade_subgrade        593994 non-null  object 
 12  loan_paid_back        593994 non-null  float64
dtypes: float64(5), int64(2), object(6)
memory usage: 58.9+ MB


In [62]:
train_df.describe()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,loan_paid_back
count,593994.000000,593994.000000,593994.000000,593994.000000,593994.000000,593994.000000,593994.000000
mean,296996.500000,48212.202976,0.120696,680.916009,15020.297629,12.356345,0.798820
std,171471.442236,26711.942078,0.068573,55.424956,6926.530568,2.008959,0.400883
min,0.000000,6002.430000,0.011000,395.000000,500.090000,3.200000,0.000000
25%,148498.250000,27934.400000,0.072000,646.000000,10279.620000,10.990000,1.000000
50%,296996.500000,46557.680000,0.096000,682.000000,15000.220000,12.370000,1.000000
75%,445494.750000,60981.320000,0.156000,719.000000,18858.580000,13.680000,1.000000
max,593993.000000,393381.740000,0.627000,849.000000,48959.950000,20.990000,1.000000


In [63]:
train_df.columns

Index(['id', 'annual_income', 'debt_to_income_ratio', 'credit_score',
       'loan_amount', 'interest_rate', 'gender', 'marital_status',
       'education_level', 'employment_status', 'loan_purpose',
       'grade_subgrade', 'loan_paid_back'],
      dtype='object')

In [64]:
input_cols = ['annual_income', 'debt_to_income_ratio', 'credit_score',
       'loan_amount', 'interest_rate', 'gender', 'marital_status',
       'education_level', 'employment_status', 'loan_purpose',
       'grade_subgrade']

target_col = 'loan_paid_back'

In [65]:
inputs = train_df[input_cols].copy()
targets = train_df[target_col].copy()

In [66]:
test_inputs = test_df[input_cols].copy()

In [67]:
numeric_cols = list(train_df[['annual_income', 'debt_to_income_ratio', 'credit_score',
       'loan_amount', 'interest_rate']])

catagorical_cols = list(train_df[['gender', 'marital_status',
       'education_level', 'employment_status', 'loan_purpose',
       'grade_subgrade']])


In [68]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

In [69]:
print(numeric_cols)

['annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate']


In [70]:
scaler = MinMaxScaler()
scaler.fit(inputs[numeric_cols])

,feature_range,"(0, ...)"
,copy,True
,clip,False


In [71]:
inputs[numeric_cols] = scaler.transform(inputs[numeric_cols])
test_inputs[numeric_cols] = scaler.transform(test_inputs[numeric_cols])

In [72]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore').fit(inputs[catagorical_cols])
encoded_cols = list(encoder.get_feature_names_out(catagorical_cols))

In [73]:
inputs[encoded_cols] = encoder.transform(inputs[catagorical_cols])
test_inputs[encoded_cols] = encoder.transform(test_inputs[catagorical_cols])

In [74]:
inputs['total_debt'] = inputs['debt_to_income_ratio'] * inputs['annual_income'] / 100
test_inputs['total_debt'] = test_inputs['debt_to_income_ratio'] * test_inputs['annual_income'] / 100

In [75]:
inputs

,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,gender_Female,gender_Male,gender_Other,marital_status_Divorced,marital_status_Married,marital_status_Single,marital_status_Widowed,education_level_Bachelor's,education_level_High School,education_level_Master's,education_level_Other,education_level_PhD,employment_status_Employed,employment_status_Retired,employment_status_Self-employed,employment_status_Student,employment_status_Unemployed,loan_purpose_Business,loan_purpose_Car,loan_purpose_Debt consolidation,loan_purpose_Education,loan_purpose_Home,loan_purpose_Medical,loan_purpose_Other,loan_purpose_Vacation,grade_subgrade_A1,grade_subgrade_A2,grade_subgrade_A3,grade_subgrade_A4,grade_subgrade_A5,grade_subgrade_B1,grade_subgrade_B2,grade_subgrade_B3,grade_subgrade_B4,grade_subgrade_B5,grade_subgrade_C1,grade_subgrade_C2,grade_subgrade_C3,grade_subgrade_C4,grade_subgrade_C5,grade_subgrade_D1,grade_subgrade_D2,grade_subgrade_D3,grade_subgrade_D4,grade_subgrade_D5,grade_subgrade_E1,grade_subgrade_E2,grade_subgrade_E3,grade_subgrade_E4,grade_subgrade_E5,grade_subgrade_F1,grade_subgrade_F2,grade_subgrade_F3,grade_subgrade_F4,grade_subgrade_F5,total_debt
0,0.060317,0.118506,0.751101,0.041856,0.588533,Female,Single,High School,Self-employed,Other,C3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000071
1,0.041576,0.251623,0.530837,0.084462,0.546374,Male,Married,Master's,Employed,Debt consolidation,D3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000105
2,0.112458,0.139610,0.658590,0.340592,0.368746,Male,Single,High School,Employed,Debt consolidation,C5,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000157
3,0.105467,0.087662,0.303965,0.086306,0.725126,Female,Single,High School,Employed,Debt consolidation,F1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000092
4,0.050323,0.068182,0.594714,0.241114,0.394042,Male,Married,High School,Employed,Other,D1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593989,0.043889,0.228896,0.678414,0.422170,0.433952,Female,Single,High School,Employed,Business,C3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000100
593990,0.075603,0.152597,0.361233,0.056896,0.641934,Male,Single,Bachelor's,Employed,Debt consolidation,F5,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000115
593991,0.106124,0.099026,0.616740,0.008856,0.614390,Female,Married,Bachelor's,Employed,Debt consolidation,C1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.

In [76]:
inputs.columns

Index(['annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount',
       'interest_rate', 'gender', 'marital_status', 'education_level',
       'employment_status', 'loan_purpose', 'grade_subgrade', 'gender_Female',
       'gender_Male', 'gender_Other', 'marital_status_Divorced',
       'marital_status_Married', 'marital_status_Single',
       'marital_status_Widowed', 'education_level_Bachelor's',
       'education_level_High School', 'education_level_Master's',
       'education_level_Other', 'education_level_PhD',
       'employment_status_Employed', 'employment_status_Retired',
       'employment_status_Self-employed', 'employment_status_Student',
       'employment_status_Unemployed', 'loan_purpose_Business',
       'loan_purpose_Car', 'loan_purpose_Debt consolidation',
       'loan_purpose_Education', 'loan_purpose_Home', 'loan_purpose_Medical',
       'loan_purpose_Other', 'loan_purpose_Vacation', 'grade_subgrade_A1',
       'grade_subgrade_A2', 'grade_subgrade_A3',

In [77]:
inputs['loan_to_income_ratio'] = inputs['loan_amount'] / (inputs['annual_income'])
test_inputs['loan_to_income_ratio'] = test_inputs['loan_amount'] / (test_inputs['annual_income'])

inputs['interest_amount'] = inputs['loan_amount'] * inputs['interest_rate'] / 100
test_inputs['interest_amount'] = test_inputs['loan_amount'] * test_inputs['interest_rate'] / 100

inputs['total_repayment'] = inputs['loan_amount'] + inputs['interest_amount']
test_inputs['total_repayment'] = test_inputs['loan_amount'] + test_inputs['interest_amount']

inputs['net_income_after_debt'] = inputs['annual_income'] - inputs['total_debt']
test_inputs['net_income_after_debt'] = test_inputs['annual_income'] - test_inputs['total_debt']

inputs['credit_to_debt_ratio'] = inputs['credit_score'] / (inputs['total_debt'] + 1)
test_inputs['credit_to_debt_ratio'] = test_inputs['credit_score'] / (test_inputs['total_debt'] + 1)


In [78]:
inputs.columns

Index(['annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount',
       'interest_rate', 'gender', 'marital_status', 'education_level',
       'employment_status', 'loan_purpose', 'grade_subgrade', 'gender_Female',
       'gender_Male', 'gender_Other', 'marital_status_Divorced',
       'marital_status_Married', 'marital_status_Single',
       'marital_status_Widowed', 'education_level_Bachelor's',
       'education_level_High School', 'education_level_Master's',
       'education_level_Other', 'education_level_PhD',
       'employment_status_Employed', 'employment_status_Retired',
       'employment_status_Self-employed', 'employment_status_Student',
       'employment_status_Unemployed', 'loan_purpose_Business',
       'loan_purpose_Car', 'loan_purpose_Debt consolidation',
       'loan_purpose_Education', 'loan_purpose_Home', 'loan_purpose_Medical',
       'loan_purpose_Other', 'loan_purpose_Vacation', 'grade_subgrade_A1',
       'grade_subgrade_A2', 'grade_subgrade_A3',

In [79]:
numeric_cols = numeric_cols + ['total_debt', 'loan_to_income_ratio', 'interest_amount', 'total_repayment', 'net_income_after_debt', 'credit_to_debt_ratio']

In [80]:
X_train = inputs[numeric_cols + encoded_cols]

In [81]:
X_train

,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,total_debt,loan_to_income_ratio,interest_amount,total_repayment,net_income_after_debt,credit_to_debt_ratio,gender_Female,gender_Male,gender_Other,marital_status_Divorced,marital_status_Married,marital_status_Single,marital_status_Widowed,education_level_Bachelor's,education_level_High School,education_level_Master's,education_level_Other,education_level_PhD,employment_status_Employed,employment_status_Retired,employment_status_Self-employed,employment_status_Student,employment_status_Unemployed,loan_purpose_Business,loan_purpose_Car,loan_purpose_Debt consolidation,loan_purpose_Education,loan_purpose_Home,loan_purpose_Medical,loan_purpose_Other,loan_purpose_Vacation,grade_subgrade_A1,grade_subgrade_A2,grade_subgrade_A3,grade_subgrade_A4,grade_subgrade_A5,grade_subgrade_B1,grade_subgrade_B2,grade_subgrade_B3,grade_subgrade_B4,grade_subgrade_B5,grade_subgrade_C1,grade_subgrade_C2,grade_subgrade_C3,grade_subgrade_C4,grade_subgrade_C5,grade_subgrade_D1,grade_subgrade_D2,grade_subgrade_D3,grade_subgrade_D4,grade_subgrade_D5,grade_subgrade_E1,grade_subgrade_E2,grade_subgrade_E3,grade_subgrade_E4,grade_subgrade_E5,grade_subgrade_F1,grade_subgrade_F2,grade_subgrade_F3,grade_subgrade_F4,grade_subgrade_F5
0,0.060317,0.118506,0.751101,0.041856,0.588533,0.000071,0.693932,0.000246,0.042102,0.060246,0.751048,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.041576,0.251623,0.530837,0.084462,0.546374,0.000105,2.031517,0.000461,0.084923,0.041471,0.530781,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.112458,0.139610,0.658590,0.340592,0.368746,0.000157,3.028628,0.001256,0.341848,0.112301,0.658487,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.105467,0.087662,0.303965,0.086306,0.725126,0.000092,0.818323,0.000626,0.086932,0.105375,0.303937,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.050323,0.068182,0.594714,0.241114,0.394042,0.000034,4.791279,0.000950,0.242064,0.050289,0.594693,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593989,0.043889,0.228896,0.678414,0.422170,0.433952,0.000100,9.618951,0.001832,0.424002,0.043789,0.678346,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
593990,0.075603,0.152597,0.361233,0.056896,0.641934,0.000115,0.752558,0.000365,0.057261,0.075488,0.361192,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
593991,0.106124,0.099026,0.616740,0.008856,0.614390,0.000105,0.083453,0.000054,0.008911,0.106019,0.616675,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,

In [82]:
X_train.columns

Index(['annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount',
       'interest_rate', 'total_debt', 'loan_to_income_ratio',
       'interest_amount', 'total_repayment', 'net_income_after_debt',
       'credit_to_debt_ratio', 'gender_Female', 'gender_Male', 'gender_Other',
       'marital_status_Divorced', 'marital_status_Married',
       'marital_status_Single', 'marital_status_Widowed',
       'education_level_Bachelor's', 'education_level_High School',
       'education_level_Master's', 'education_level_Other',
       'education_level_PhD', 'employment_status_Employed',
       'employment_status_Retired', 'employment_status_Self-employed',
       'employment_status_Student', 'employment_status_Unemployed',
       'loan_purpose_Business', 'loan_purpose_Car',
       'loan_purpose_Debt consolidation', 'loan_purpose_Education',
       'loan_purpose_Home', 'loan_purpose_Medical', 'loan_purpose_Other',
       'loan_purpose_Vacation', 'grade_subgrade_A1', 'grade_subgrade_A2

In [83]:
from lightgbm import LGBMClassifier

In [84]:
model = LGBMClassifier(random_state=42, n_estimators=1000, learning_rate=0.05, num_leaves=31, colsample_bytree=0.8, subsample=0.8, max_depth=7).fit(X_train, targets)

In [85]:
train_preds = model.predict(X_train)

In [86]:
train_preds

array([1., 1., 1., ..., 1., 1., 1.], shape=(593994,))

In [87]:
from sklearn.metrics import roc_curve, roc_auc_score

In [88]:
train_pred_proba = model.predict_proba(X_train)[:,1]
auc = roc_auc_score(targets, train_pred_proba)
print(f'Training AUC: {auc}')

Training AUC: 0.9327242148795616


In [89]:
X_train, X_val, train_targets, val_targets = train_test_split(X_train, targets, test_size=0.2, random_state=42)

In [90]:
model = LGBMClassifier(random_state=42, n_estimators=1000, learning_rate=0.05, num_leaves=31, colsample_bytree=0.8, subsample=0.8, max_depth=7).fit(X_train, train_targets)

KeyboardInterrupt: 

In [ ]:
train_pred_proba = model.predict_proba(X_train)[:,1]
auc = roc_auc_score(train_targets, train_pred_proba)
print(f'Training AUC: {auc}')

Training AUC: 0.9345283100287106


In [ ]:
val_pred_proba = model.predict_proba(X_val)[:,1]
auc = roc_auc_score(val_targets, val_pred_proba)
print(f'Validation AUC: {auc}')

Validation AUC: 0.9231247267159564


In [ ]:
submission_df = pd.read_csv('sample_submission.csv')
submission_df

,id,loan_paid_back
0,593994,0
1,593995,0
2,593996,0
3,593997,0
4,593998,0
...,...,...
254564,848558,0
254565,848559,0
254566,848560,0
254567,848561,0


In [ ]:
test_preds = model.predict_proba(test_inputs[numeric_cols + encoded_cols])[:,1]
submission_df.loan_paid_back = test_preds
submission_df.to_csv('submission.csv', index=False)

In [ ]:
submission_df.head(10)

,id,loan_paid_back
0,593994,0.931967
1,593995,0.983282
2,593996,0.496709
3,593997,0.915385
4,593998,0.950129
5,593999,0.966565
6,594000,0.985099
7,594001,0.978108
8,594002,0.944280
9,594003,0.002729


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
models = {
    "XGB": XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, subsample=0.8, random_state=42),
    "LGBM": LGBMRegressor(n_estimators=500, learning_rate=0.05, subsample=0.8, random_state=42)
}

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error

In [ ]:
preds = []
rmses = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for name, model in models.items():
    print(f"Training {name}...")
    fold_preds = np.zeros(len(X_train))

    for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
        X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_tr, y_val = train_targets.iloc[train_index], train_targets.iloc[val_index]
        
        model.fit(X_tr, y_tr)
        val_pred = model.predict(X_val)
        
        rmse = root_mean_squared_error(y_val, val_pred)
        rmses.append(rmse)
        fold_preds[val_index] = val_pred

        fold_rmse = root_mean_squared_error(y_val, val_pred)
        print(f"    Fold{fold + 1} : RMSE = {fold_rmse:.5f}")
    
    overall_rmse = root_mean_squared_error(train_targets, fold_preds)
    print(f"{name} Overall RMSE: {overall_rmse:.5f}")

    preds.append(fold_preds)
    rmses.append(overall_rmse)

Training LinearRegression...
    Fold1 : RMSE = 0.28305
    Fold2 : RMSE = 0.28078
    Fold3 : RMSE = 0.27963
    Fold4 : RMSE = 0.28034
    Fold5 : RMSE = 0.27966
LinearRegression Overall RMSE: 0.28070
Training Ridge...
    Fold1 : RMSE = 0.28317
    Fold2 : RMSE = 0.28091
    Fold3 : RMSE = 0.27982
    Fold4 : RMSE = 0.28057
    Fold5 : RMSE = 0.27984
Ridge Overall RMSE: 0.28087
Training Lasso...
    Fold1 : RMSE = 0.28385
    Fold2 : RMSE = 0.28147
    Fold3 : RMSE = 0.28044
    Fold4 : RMSE = 0.28119
    Fold5 : RMSE = 0.28042
Lasso Overall RMSE: 0.28148
Training ElasticNet...
    Fold1 : RMSE = 0.28355
    Fold2 : RMSE = 0.28120
    Fold3 : RMSE = 0.28018
    Fold4 : RMSE = 0.28090
    Fold5 : RMSE = 0.28015
ElasticNet Overall RMSE: 0.28120
Training XGB...
    Fold1 : RMSE = 0.27180
    Fold2 : RMSE = 0.27019
    Fold3 : RMSE = 0.26883
    Fold4 : RMSE = 0.26922
    Fold5 : RMSE = 0.26916
XGB Overall RMSE: 0.26984
Training LGBM...
[LightGBM] [Warning] Found whitespace in feature_n

In [ ]:
inv_rmse = 1 / np.array(rmses)
weights = inv_rmse / inv_rmse.sum()
final_pred = np.sum([w * p for w, p in zip(weights, preds)], axis = 0)

ensemble_rmse = root_mean_squared_error(train_targets, final_pred)
print("\nWeighted Ensemble RMSE:", ensemble_rmse)
print("Model Weights:", dict(zip(models.keys(), weights)))


Weighted Ensemble RMSE: 0.7623398287662324
Model Weights: {'LinearRegression': np.float64(0.027192121916656757), 'Ridge': np.float64(0.02741170087075278), 'Lasso': np.float64(0.027524292802856565), 'ElasticNet': np.float64(0.02745467690148328), 'XGB': np.float64(0.02752135026665747), 'LGBM': np.float64(0.027420009921355893)}


In [ ]:
X = X_train
y = train_targets

In [ ]:
base_models = {     
    "LinearRegression" : LinearRegression(),
    "Ridge": Ridge(alpha=1.0, random_state=42, max_iter=1000),
    "Lasso": Lasso(alpha=0.001, max_iter=1000, random_state=42),
    "ElasticNet": ElasticNet(alpha=0.001, l1_ratio=0.5, random_state= 42),
    "XGB": XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, subsample=0.8, random_state=42),
    "LGBM": LGBMRegressor(n_estimators=500, learning_rate=0.05, subsample=0.8, random_state=42)
}


In [ ]:
meta_model = LGBMRegressor(n_estimators=500, learning_rate=0.05, subsample=0.8, random_state=42)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
oof_preds = np.zeros((X.shape[0], len(base_models)))

In [ ]:
for i, (name, model) in enumerate(base_models.items()):
    print(f"Training base model: {name}")
    oof_pred = np.zeros(X.shape[0])
    for train_idx, val_index in kf.split(X):
        X_tr, X_val = X.iloc[train_index], X.iloc[val_index]
        y_tr, y_val = y.iloc[train_index], y.iloc[val_index]
        
        model.fit(X_tr, y_tr)
        val_pred = model.predict(X_val)
        oof_pred[val_index] = val_pred

    oof_preds[:, i] = oof_pred
    rmse = root_mean_squared_error(y, oof_pred)
    print(f"{name} RMSE: {rmse:.5f}")

Training base model: LinearRegression
LinearRegression RMSE: 0.28066
Training base model: Ridge
Ridge RMSE: 0.28083
Training base model: Lasso
Lasso RMSE: 0.28147
Training base model: ElasticNet
ElasticNet RMSE: 0.28118
Training base model: XGB
XGB RMSE: 0.26291
Training base model: LGBM
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026649 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2914
[LightGBM] [Info] Number of data points in the train set: 380156, number of used features: 66
[LightGBM] [Info] Start training from score 0.798948
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.023233 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2914
[LightGBM] [

In [ ]:
meta_model.fit(oof_preds, y)

,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


In [ ]:
X_test = test_inputs[numeric_cols + encoded_cols]
test_preds = np.zeros((X_test.shape[0], len(base_models)))

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for i, (name, model) in enumerate(base_models.items()):
    print(f"Generating test predictions for base model: {name}")
    fold_test_preds = np.zeros(X_test.shape[0])

    for train_idx, val_index in kf.split(X):
        X_tr, X_val = X.iloc[train_index], X.iloc[val_index]
        y_tr, y_val = y.iloc[train_index], y.iloc[val_index]
        
        model.fit(X_tr, y_tr)
        fold_test_preds += model.predict(X_test) / kf.n_splits

        test_preds[:, i] = fold_test_preds


Generating test predictions for base model: LinearRegression
Generating test predictions for base model: Ridge
Generating test predictions for base model: Lasso
Generating test predictions for base model: ElasticNet
Generating test predictions for base model: XGB
Generating test predictions for base model: LGBM
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011023 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2914
[LightGBM] [Info] Number of data points in the train set: 380156, number of used features: 66
[LightGBM] [Info] Start training from score 0.798948
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.023233 seconds.
You can set `for

In [ ]:
final_test_preds = meta_model.predict(test_preds)

In [ ]:
submission_df = pd.read_csv('sample_submission.csv')
submission_df.loan_paid_back = final_test_preds
submission_df.to_csv('stacked_submission.csv', index=False)

In [110]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import plotly.express as px
from scipy import stats
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve

base_models = {
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "DecisionTree": DecisionTreeClassifier(max_depth=7, random_state=42),
    "XGB" : XGBClassifier(n_estimators=800,        # number of trees (boosting rounds)
    learning_rate=0.03,      # smaller = slower but more accurate
    max_depth=6,             # depth of each tree
    subsample=0.8,           # % of rows used per tree
    colsample_bytree=0.8,    # % of features used per tree
    min_child_weight=3,      # prevents overfitting (like min_samples_leaf)
    gamma=0.1,               # minimum loss reduction to make a split
    reg_lambda=1.0,          # L2 regularization
    reg_alpha=0.0,           # L1 regularization
    objective='binary:logistic',
    eval_metric='auc',
    random_state=42,
    n_jobs=-1    ),
    "LGBMClassifier" : LGBMClassifier(
    n_estimators=1000,       # more trees, smaller learning rate
    learning_rate=0.02,
    num_leaves=31,           # controls tree complexity (similar to max_depth)
    max_depth=-1,            # -1 means no limit
    subsample=0.8,           # % of rows per tree
    colsample_bytree=0.8,    # % of features per tree
    min_child_samples=50,    # min samples per leaf (overfitting control)
    reg_lambda=1.0,          # L2 regularization
    reg_alpha=0.0,           # L1 regularization
    objective='binary',
    metric='auc',
    random_state=42,
    n_jobs=-1
    )
}

X = X_train
y = train_targets

In [111]:
meta_model = LGBMClassifier(n_estimators=500, learning_rate=0.05, subsample=0.8, random_state=42)

In [112]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros((X_train.shape[0], len(base_models)))

In [117]:
for i, (name, model) in enumerate(base_models.items()):
    print(f"Training base model: {name}")
    oof_pred = np.zeros(X.shape[0])
    for train_idx, val_idx in kf.split(X):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_tr, y_tr)
        val_pred_proba = model.predict_proba(X_val)[:, 1]
        oof_pred[val_idx] = val_pred_proba

    oof_preds[:, i] = oof_pred
    roc = roc_auc_score(y, oof_pred)
    print(f"{name} : {roc}")

Training base model: LogisticRegression
LogisticRegression : 0.9102999463985016
Training base model: DecisionTree
DecisionTree : 0.909399708284303
Training base model: XGB
XGB : 0.9199422285918385
Training base model: LGBMClassifier
LGBMClassifier : 0.9204207178381068


In [118]:
meta_model.fit(oof_preds, y)

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.05
,n_estimators,500
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [123]:
X_test = test_inputs[numeric_cols + encoded_cols]
test_preds = np.zeros((X_test.shape[0], len(base_models)))

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for i, (name, model) in enumerate(base_models.items()):
    print(f"Generating test predictions for base model: {name}")
    fold_test_preds = np.zeros(X_test.shape[0])

    for train_idx, val_index in kf.split(X):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_index]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_index]
        
        model.fit(X_tr, y_tr)
        fold_test_preds += model.predict_proba(X_test)[:, 1] / kf.n_splits

    test_preds[:, i] = fold_test_preds
meta_model.fit(oof_preds, y)
meta_test_preds = meta_model.predict_proba(test_preds)[:, 1]


Generating test predictions for base model: LogisticRegression
Generating test predictions for base model: DecisionTree
Generating test predictions for base model: XGB
Generating test predictions for base model: LGBMClassifier


c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [124]:
submission_df = pd.read_csv('sample_submission.csv')
submission_df

,id,loan_paid_back
0,593994,0
1,593995,0
2,593996,0
3,593997,0
4,593998,0
...,...,...
254564,848558,0
254565,848559,0
254566,848560,0
254567,848561,0


In [125]:
submission_df.loan_paid_back = meta_test_preds
submission_df

,id,loan_paid_back
0,593994,0.925880
1,593995,0.971929
2,593996,0.486160
3,593997,0.903319
4,593998,0.962161
...,...,...
254564,848558,0.993595
254565,848559,0.783280
254566,848560,0.968076
254567,848561,0.985444


In [130]:
submission_df.to_csv('new_submission.csv', index= None)

In [ ]:
import optuna
